In [1]:
import pandas as pd
import numpy as np

# Load

In [15]:
df_reddit = pd.read_csv('./ready/reddit_vaccine_emotions.csv')
df_guardian = pd.read_csv('./ready/guardian_vaccine_emotions.csv')
df_nyt = pd.read_csv('./ready/nyt_vaccine_emotions.csv')

# Col Drop

In [16]:
emotion_28 = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
    'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
    'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

reddit = ['created_utc', 'text_clean']
guardian = ['pub_date', 'text_clean']
nyt = ['pub_date', 'text_clean']

In [17]:
df_reddit = df_reddit[emotion_28 + reddit]
df_guardian = df_guardian[emotion_28 + guardian]
df_nyt = df_nyt[emotion_28 + nyt]

# col rename, reddit ~2018 drop

In [18]:
# datetime 변환 (timezone-aware 유지)
df_reddit['created_utc'] = pd.to_datetime(df_reddit['created_utc'], utc=True, errors='coerce')

# 2019-01-01 기준으로 필터링 (UTC-aware로 비교)
cutoff = pd.Timestamp('2019-01-01', tz='UTC')
df_reddit = df_reddit[df_reddit['created_utc'] >= cutoff]

# col rename
df_reddit.rename(columns = {'created_utc': 'pub_date'}, inplace=True)

# col reorder
new_order = ['pub_date', 'text_clean'] + emotion_28
df_reddit = df_reddit[new_order]

In [19]:
# 변환
df_guardian['pub_date'] = pd.to_datetime(df_guardian['pub_date'], errors='coerce').dt.tz_localize(None)
df_nyt['pub_date'] = pd.to_datetime(df_nyt['pub_date'], errors='coerce').dt.tz_localize(None)

# 공적 미디어 레이블 추가 (선택)
df_guardian['platform'] = 'Guardian'
df_nyt['platform'] = 'NYT'

# 두 데이터 합치기
df_public = pd.concat([df_guardian, df_nyt], ignore_index=True)

# 데이터프레임 컬럼 재정렬
new_order = ['pub_date', 'text_clean', 'platform'] + emotion_28
df_public = df_public[new_order]

In [20]:
# save
df_public.to_csv('./ready/public_media_vaccine.csv', index=False, encoding='utf-8-sig')
df_reddit.to_csv('./ready/civil_platform_vaccine.csv', index=False, encoding='utf-8-sig')